In [ ]:
import json
import pandas as pd
import re
import hashlib
import time
import os
from tqdm import tqdm
import re
import sys
from config import japanese_grammar

In [ ]:
import text_to_audio

In [ ]:
def chinese_to_number(chinese_num):
    num_dict = {
        '零': 0, '一': 1, '二': 2, '三': 3, '四': 4,
        '五': 5, '六': 6, '七': 7, '八': 8, '九': 9
    }

    unit_dict = {
        '十': 10, '百': 100, '千': 1000
    }

    result = 0
    unit = 1  # 当前的单位，初始为1
    num = 0  # 当前的数字，初始为0
    length = len(chinese_num)

    for i in range(length):
        char = chinese_num[i]

        if char in num_dict:
            num = num_dict[char]

            if i == length - 1:
                result += num * unit

        elif char in unit_dict:
            unit = unit_dict[char]

            if i == 0 or chinese_num[i - 1] not in num_dict:
                num = 1

            result += num * unit
            unit = 1
            num = 0

    return result

# 测试样例
print(chinese_to_number("一百二十九"))  # 129
print(chinese_to_number("九百九十九"))  # 999
print(chinese_to_number("五十三"))      # 53
print(chinese_to_number("五"))          # 5

In [ ]:
def get_md5(content):
    content = content.encode("utf-8")
    md5_hash = hashlib.md5()
    md5_hash.update(content)
    return md5_hash.hexdigest()

In [ ]:
kanji_with_kana_pattern = re.compile(
    r"\s*(?P<kanji>[\u4E00-\u9FFF]) (?P<kana>[\u3040-\u309F]+)\s*"
)


def transform_content_to_ruby_rich_text(content, pattern=None):
    if pattern is None:
        pattern = kanji_with_kana_pattern

    return pattern.sub(r"<ruby>\1<rt>\2</rt></ruby>", content)


def split_kanji_and_its_kana(content, pattern=None):
    if pattern is None:
        pattern = kanji_with_kana_pattern

    return kanji_with_kana_pattern.sub(r"\1", content), kanji_with_kana_pattern.sub(
        r"\2", content
    )

In [ ]:
"其中N1级别201个，N2级别150个，N3级别140个，N4级别130个，N5级别120个。"

import json
import re

content = """



89. ～とて


（1）

接続

名＋とて

説明

表示假定条件的逆接，“即使……也不例外”“即使是……也……”“甚至……”“就连……”。

例文

△常に冷静な彼とて やはり人間だから、感情的になってしまうこともあるのだろう。 【2009年12月真题】/尽管他平时很冷静，但是也有情绪化的时候吧 。

△最近の電気製品は機能が多すぎる。開発者たちとて すべての機能が必要とは思わないのではないか。 【2007年真题】/最近的电器功能实在太多，就算是开发商也未必会认为所有的功能都有必要吧 。

△私とて 試合に負けたことに悔しい。 /输掉了比赛，我也很懊恼 。

注意

①前面主要接续人名。

②意思与「～としても」 相同，但「～とて」 是比较生硬的表达方式。

（2）

接続

名＋だ/動た形＋とて

説明

表示假定条件的逆接，无论前项怎样，后项的原则都是一样的。“即使……也……”“即便……也……”“就算……也……”。

例文

△たとえ病気だとて 試験に欠席してはいけない。 /就算生病了也不能缺考 。

△いくら 頼 たの んだとて 、できないことはできない。 /再怎么拜托，不能做的事情就是不能做 。

△どんなに後悔したとて 、過ぎたことは今さらどうしようもない。 /就算再怎么后悔，对于已经过去的事情也没有办法 。

注意

常与「たとえ」 「どんなに」 「いくら」 等词一起使用。




"""

level_pattern = re.compile(r"^N([1-5])文法")
unit_pattern = re.compile(r"^第(\d+)单元(.*)")
grammar_start_pattern = re.compile(r"^\d+\. ")
seq_pattern = re.compile(r"^（\d+）")


def get_japanese_sequence_sign(number):
    number = int(number)
    if number <= 0:
        return "⓪"

    return chr(ord("\u2460") + number - 1)

def merge_example_sentences(sentences):
    new_sentences = []
    for sentence in sentences:
        if not sentence.startswith("△"):
            new_sentences[-1] += "\n" + sentence
        elif sentence.startswith("△B"):
            new_sentences[-1] += "\n  " + sentence[1:]
        else:
            new_sentences.append(sentence)

    return new_sentences


def extract_grammars(content):
    grammars = {}
    lines = []
    part_keywords = {'接続': 391, '説明': 401, '例文': 910, '注意': 774, '接续': 308, '说明': 515, '读法': 4, '例词': 2, '补充': 1}
    keyword_mapping = {
        '接続': "接续",
        '説明': "说明",
        '读法': "说明",
        '例词': "说明",
        '补充': "注意",
    }
    level = 0
    current_grammar = ""
    current_part = ""
    current_seq = ""
    for idx, line in enumerate(content.split("\n")):
        line = line.strip()
        if not line:
            continue

        line = line.replace("真题】", "JLPT】")
        if res := level_pattern.findall(line):
            level = int(res[0])
            # print(f"Current Level: {level}")
            continue

        if res := unit_pattern.findall(line):
            # print(f"Current Unit: {res[0][0]}.{res[0][1]}")
            continue

        if grammar_start_pattern.match(line):
            # print(f"Current Grammar: {line}")
            current_grammar = line
            grammars[current_grammar] = {}
            grammars[current_grammar]["level"] = level
            # current_part = ""
            # current_seq = ""
            continue

        if seq_pattern.match(line):
            print(f"Current Seq: {line}")
            current_seq = line
            continue

        lines.append(line)

        if len(line) < 5 and line[:2] in part_keywords:
            current_part = line[:2]
            current_part = keyword_mapping.get(current_part, current_part)
            index = f"{get_japanese_sequence_sign(line[2])} " if len(line) > 2 else ""
            if current_part not in grammars[current_grammar]:
                if current_part == "例文":
                    grammars[current_grammar][current_part] = []
                else:
                    grammars[current_grammar][current_part] = ""
            continue

        try:
            if current_part == "例文":
                grammars[current_grammar][current_part].append(line)
            else:
                grammars[current_grammar][current_part] += index + line + "\n"
                index = ""
        except Exception:
            print((idx, line))
            raise

    return grammars


In [ ]:
with open("grammars_N1-N5.json", "r", encoding="utf-8") as fp:
    df = pd.DataFrame(json.loads(fp.read()))


df_res = df
keyword = "に"
for item in keyword.split():
    df_res = df_res[df_res.content.str.replace(r"[（）()\s]", "", regex=True).str.contains(item)
                    | df_res.hiragana.str.replace(r"[（）()\s]", "", regex=True).str.contains(item)
                    | df_res.chinese_meaning.str.contains(item) | (df_res.source == item)]
df_res["order"] = df_res.content.apply(len)
df_res = df_res.sort_values(by="order")
[df.loc[index].to_dict() for index in df_res.index]

In [ ]:
df = pd.read_json('grammars_N1-N5.json').fillna("")
df.level = df.level.apply(lambda x: f"N{x}")
df.example = df.example.apply(lambda example: "<br>".join([f"{sentence['content']}{sentence['tag']}<br>{sentence['meaning']}<br>" for sentence in example]))
print(df.columns)
print(df.level.unique())
print(df.source.unique())
df = df[["content", "hiragana", "meaning", "usage", "remark", "level", "example"]]
df.to_csv("grammars_N1-N5_anki.csv", index=False, header=False)

In [ ]:
kanji_with_kana_pattern = re.compile(
    r"(?P<kanji>[\u4E00-\u9FFF])(?P<kana>[\u3040-\u309F]+)(?:\s|$)"
)
content = """
▶ あかい「赤い」 ②⓪（形）：红色的

解 古代日本人不区分“红”和“亮”两个概念，因为明亮的太阳光和火光都偏红色。

例 赤あか い花はな が咲さ いている。/红色的花开了。
"""

print(transform_content_to_ruby_rich_text(content, pattern=kanji_with_kana_pattern))

In [ ]:
kana_meaning_pattern = re.compile(r"^\d{2,}\s")
example_pattern = re.compile(r"^▶")

result = []
kanji_with_kana_pattern = re.compile(
    r"(?P<kanji>[\u4E00-\u9FFF]+)(?P<kana>[\u3040-\u309F]+)(?:\s|$)"
)
file_path = (
    japanese_grammar.book_folder
    / "记单词，一定要学的130个日语词根-_张铭_-_Z-Library_.txt"
)
with open(file_path, "r", encoding="utf-8") as fp:
    current_kana = ""
    current_meaning_item = ""
    current_meanning_explain = []
    current_example = ""
    current_example_explain = []
    within_example_part = False

    lines = fp.readlines()
    last_line = ""
    for line in lines:
        content = line.strip()
        if not content:
            continue

        if current_kana and content.startswith("第二章"):
            break

        # 假名
        if len(content) == 1:
            current_kana = content
            current_meaning_item = ""
            current_meanning_explain = []
            current_example = ""
            current_example_explain = []
            within_example_part = False
            continue

        # 假名词根含义
        if current_kana and kana_meaning_pattern.match(content):
            if current_example:
                result.append(
                    {
                        "kana": current_kana,
                        "meaning": current_meaning_item,
                        "explain": "<br>".join(current_meanning_explain),
                        "example": current_example,
                        "example_explain": "<br>".join(
                            [
                                transform_content_to_ruby_rich_text(
                                    content, pattern=kanji_with_kana_pattern
                                )
                                for content in current_example_explain
                            ]
                        ),
                    }
                )

            current_meaning_item = content
            current_meanning_explain = []
            current_example = ""
            within_example_part = False
            item_idx = content.split()[0]
            continue

        # 词根含义举例
        if current_kana and example_pattern.match(content):
            within_example_part = True
            if current_example:
                result.append(
                    {
                        "kana": current_kana,
                        "meaning": current_meaning_item,
                        "explain": "<br>".join(current_meanning_explain),
                        "example": current_example,
                        "example_explain": "<br>".join(
                            [
                                transform_content_to_ruby_rich_text(
                                    content, pattern=kanji_with_kana_pattern
                                )
                                for content in current_example_explain
                            ]
                        ),
                    }
                )

            current_example = content
            current_example_explain = []
            continue

        # 词根举例 解释
        if current_example:
            current_example_explain.append(content)
            continue

        # 假名词根 解释
        if current_kana and not within_example_part:
            current_meanning_explain.append(content)

        last_line = content


if current_example_explain:
    result.append(
        {
            "kana": current_kana,
            "meaning": current_meaning_item,
            "explain": "<br>".join(current_meanning_explain),
            "example": current_example,
            "example_explain": "<br>".join(
                [
                    transform_content_to_ruby_rich_text(
                        content, pattern=kanji_with_kana_pattern
                    )
                    for content in current_example_explain
                ]
            ),
        }
    )

In [ ]:
for record in result:
    if record["example"] == "▶ あか「垢」 ②（名）：污垢；水锈":
        print(record)
        break

In [ ]:
df = pd.DataFrame(result).fillna("")
print(df.shape)
df.to_csv("word_roots.csv", index=False)
df.to_csv("word_roots_anki.csv", index=False, header=False)
df.head()


In [ ]:
df.head()

In [ ]:
last_kana = ""
last_meaning = ""
for index in df.index:
    detail = df.loc[index].to_dict()
    if detail["kana"] != last_kana:
        print("=" * 32)
        print(detail["kana"])
    if detail["meaning"] != last_meaning:
        print()
        print(detail["meaning"])
        print(detail["explain"])

    example_explain = detail["example_explain"].split("<br>")[0]
    example_explain = "" if example_explain[0] != "解" else " ".join(example_explain.split()[1:])
    print(detail["example"])
    if example_explain:
        print(example_explain)

    last_kana = detail["kana"]
    last_meaning = detail["meaning"]

In [ ]:
result_type_answer = []

kana_pattern = re.compile(r"(?<=^▶ )([\u3040-\u309F]+)(?=「)")
meaning_pattern = re.compile(r"(?<=：)(.+)$")
content_pattern = re.compile(r"^▶ ([\u3040-\u309F]+)(?:「(.+)」|\s?」)?\s*([⓪\u2460-\u2470]+)(（[·、\u4E00-\u9FFFサ]+）)：(.*)$")
"""
▶ あかい「赤い」 ②⓪（形）：红色的
▶ あける「明ける」 ⓪（自一）：天亮了；过年
▶ {{c1::あかい}}「赤い」 ②⓪（形）：{{c2::红色的}}
"""
for record in result:
    result_type_answer.append({
        "word": content_pattern.sub(r"{{c1::\1}}<br>\2 \3 \4<br>{{c2::\5}}", record["example"]),
        "word_explain": record["example_explain"],
        "kana_meaning": record["meaning"],
        "kana_meaning_explain": record["explain"],
    })

df_res = pd.DataFrame(result_type_answer, dtype=str).fillna("")
df_res.to_csv("word_roots(type_answer_anki).csv", index=False, header=False)
print(df_res.shape)
df_res.head()

In [ ]:
import pandas as pd
import re

sentence_content_pattern = re.compile(r'([\u2460-\u2470])\s*')
invalid_line_pattern = re.compile(r'^[\d\s]+$')

example_sentences = []
groups = []
with open("CS_Word_20240405_22.25.17.txt", "r", encoding="utf-8") as fp:
    matched = False
    sentence = None
    last_line = ""
    for line in fp.readlines():
        line = line.strip()
        if len(line) < 3 or invalid_line_pattern.match(line.strip()):
            continue

        if matched == True and last_line[-1] != "。":
            sentence["content"] += line
            last_line = line
            continue

        if sentence_content_pattern.match(line.strip()):
            matched = True
            sentence = {"content": sentence_content_pattern.sub(r"\1 ", line)}
        else:
            if matched:
                sentence["hiragana"] = ""
                sentence["meaning"] = line
                if sentence["meaning"][1] == " ":
                    sentence["meaning"] = sentence["meaning"][2:]

                if sentence["meaning"][0] == "囉":
                    sentence["meaning"] = sentence["meaning"][1:]

                if sentence["content"][0] == "①" and groups:
                    example_sentences.append(groups)
                    groups = []

                groups.append(sentence)
            matched = False
        last_line = line

if groups:
    example_sentences.append(groups)

print(example_sentences[1])
with open("example_sentences.json", "w", encoding="utf-8") as fp:
    fp.write(json.dumps(example_sentences, ensure_ascii=False, indent=4))
    # pd.DataFrame(example_sentences, dtype=str).fillna("").to_json(fp, orient="records", force_ascii=False, indent=4)

In [ ]:
import json
import pandas as pd

df = pd.read_json("grammar_pd.json", dtype=str)
with open("grammar.json", "r", encoding="utf-8") as f:
    data = json.loads(f.read())
    df = pd.DataFrame(data)
df.head()
detail = df.loc[0].to_dict()
print(type(detail["example"]))
print(detail["example"])

In [ ]:
from grammar_enumeration import grammars, df_grammars

In [ ]:
df_grammars.shape

In [ ]:
df_grammars.hiragana.str.replace(r"[（）()\s()]", "").str.contains(r'なしに')[23]

In [ ]:
keyword = "なくして(は)"
df = df_grammars
df[df.content.str.replace(r"[（）()\s()]", "").str.contains(keyword)
                    | df.hiragana.str.replace(r"[（）()\s()]", "").str.contains(keyword)
                    | df.chinese_meaning.str.contains(keyword) | (df.source == keyword)]

In [ ]:
"其中N1级别201个，N2级别150个，N3级别140个，N4级别130个，N5级别120个。"

import json
import re

level_pattern = re.compile(r"^N([1-5])文法")
unit_pattern = re.compile(r"^第(\d+)单元(.*)")
grammar_start_pattern = re.compile(r"^\d+\. ")
seq_pattern = re.compile(r"^（\d+）")
from collections import defaultdict

lines = []
part_keywords = {'接続': 391, '説明': 401, '例文': 910, '注意': 774, '接续': 308, '说明': 515, '读法': 4, '例词': 2, '补充': 1}
level = 0
current_grammar = ""
current_part = ""
current_seq = ""
output = False
# with open("日语蓝宝书.txt", "r", encoding="utf-8") as fp:
# for idx, line in enumerate(fp.readlines()):
content = """



89. ～とて


（1）

接続

名＋とて

説明

表示假定条件的逆接，“即使……也不例外”“即使是……也……”“甚至……”“就连……”。

例文

△常に冷静な彼とて やはり人間だから、感情的になってしまうこともあるのだろう。 【2009年12月真题】/尽管他平时很冷静，但是也有情绪化的时候吧 。

△最近の電気製品は機能が多すぎる。開発者たちとて すべての機能が必要とは思わないのではないか。 【2007年真题】/最近的电器功能实在太多，就算是开发商也未必会认为所有的功能都有必要吧 。

△私とて 試合に負けたことに悔しい。 /输掉了比赛，我也很懊恼 。

注意

①前面主要接续人名。

②意思与「～としても」 相同，但「～とて」 是比较生硬的表达方式。

（2）

接続

名＋だ/動た形＋とて

説明

表示假定条件的逆接，无论前项怎样，后项的原则都是一样的。“即使……也……”“即便……也……”“就算……也……”。

例文

△たとえ病気だとて 試験に欠席してはいけない。 /就算生病了也不能缺考 。

△いくら 頼 たの んだとて 、できないことはできない。 /再怎么拜托，不能做的事情就是不能做 。

△どんなに後悔したとて 、過ぎたことは今さらどうしようもない。 /就算再怎么后悔，对于已经过去的事情也没有办法 。

注意

常与「たとえ」 「どんなに」 「いくら」 等词一起使用。




"""
grammars = {}
for _ in range(1):
    for idx, line in enumerate(content.split("\n")):
        line = line.strip()
        if not line:
            continue

        if res := level_pattern.findall(line):
            level = int(res[0])
            # print(f"Current Level: {level}")
            continue

        if res := unit_pattern.findall(line):
            # print(f"Current Unit: {res[0][0]}.{res[0][1]}")
            continue

        if grammar_start_pattern.match(line):
            print(f"Current Grammar: {line}")
            current_grammar = line
            grammars[current_grammar] = {}
            # current_part = ""
            # current_seq = ""
            continue

        if seq_pattern.match(line):
            # print(f"Current Seq: {line}")
            current_seq = line
            continue

        lines.append(line)

        if len(line) < 5 and line[:2] in part_keywords:
            current_part = line[:2]
            if current_part == "例文":
                grammars[current_grammar][current_part] = []
            else:
                grammars[current_grammar][current_part] = ""
            continue

        if current_part == "例文":
            grammars[current_grammar][current_part].append(line)
        else:
            grammars[current_grammar][current_part] += line + "\n"

print(json.dumps(grammars, ensure_ascii=False, indent=4))

In [ ]:
json.dumps("⓪①②③")

In [ ]:
chr(ord("\u2460"))

In [ ]:
kanji_with_kana_pattern = re.compile(r"(?P<kanji>[\u4E00-\u9FFF]) (?P<kana>[\u3040-\u309F]+) ")
content = "中 なか 川 がわ 先 せん 生 せい 「あ、 山 やま 口 ぐち さん。 偶 ぐう 然 ぜん ですね。」"
content = "辞书形 ⇒ 尊他语(辞书形)"
content = "做 | する ⇒ なさる"
content = "84. ～ 中 ちゅう/じゅう"
print(kanji_with_kana_pattern.findall(content))
kanji_with_kana_pattern.sub(r"<ruby>\1<rt>\2</rt></ruby>", content)

In [ ]:


content = "△お 客 きゃく 様 さま 、このお 皿 さら をさげてもよろしいでしょうか 。 /客人您好，这个盘子我可以撤下去了吗 ？"
print(transform_content_to_ruby_rich_text(content))


print(split_kanji_and_its_kana(content))

In [ ]:
with open(japanese_grammar.book_folder / "日语蓝宝书.txt", "r", encoding="utf-8") as fp:
    content = fp.read()

empty_usage = []
grammars = extract_grammars(content)

len(grammars)

In [ ]:
import pandas as pd
df = pd.DataFrame(grammars.values())
df.head()

In [ ]:
with open(japanese_grammar.book_folder / "日语蓝宝书.txt", "r", encoding="utf-8") as fp:
    content = fp.read()

empty_usage = []
grammars = extract_grammars(content)
# print(json.dumps(grammars, ensure_ascii=False, indent=4))
part_keywords = {'接続': 391, '説明': 401, '例文': 910, '注意': 774, '接续': 308, '说明': 515, '读法': 4, '例词': 2, '补充': 1}
example_sentence_pattern = re.compile(r"^([^/]+?)(【\d+年(?:\d+月)?JLPT】)?/(.*)$", re.DOTALL)
header_pattern = re.compile(r"^△\s*")
formatted_grammars = []
for idx, (grammar, detail) in enumerate(grammars.items()):
    try:
        content = grammar_start_pattern.sub("", grammar)
        hiragana = ""
        rich_text = transform_content_to_ruby_rich_text(content)
        if rich_text != content:
            content, hiragana = split_kanji_and_its_kana(content)

        example_sentences = detail.get("例文") or []
        example_sentences = merge_example_sentences(example_sentences)
        formated_exmpale_sentences = []

        for sentence_seq, sentence in enumerate(example_sentences):
            sentence_content, tag, meaning = example_sentence_pattern.findall(sentence)[0]
            sentence_content = header_pattern.sub("", sentence_content)
            sentence_content = f"{get_japanese_sequence_sign(sentence_seq+1)} {sentence_content}"

            sentence_content = transform_content_to_ruby_rich_text(sentence_content)
            formated_exmpale_sentences.append({"content": sentence_content, "tag": tag, "meaning": meaning})

        # example_sentences = [{"content": sentence} for sentence in map(transform_content_to_ruby_rich_text, detail.get("例文") or [])]

        formatted_grammars.append({
            "id": idx + 1,
            "content": content,
            "hiragana": hiragana,
            "meaning": detail.get("说明", "").strip(),
            "usage": transform_content_to_ruby_rich_text(detail.get("接续", "").strip()),
            "example": formated_exmpale_sentences,
            "remark": transform_content_to_ruby_rich_text(detail.get("注意", "").strip()),
            "source": "S2N1-N5",
            "japanese_meaning": "",
            "chinese_meaning": detail.get("说明", "").strip(),
            "level": detail["level"]
        })
        if formatted_grammars[-1]["usage"] and formatted_grammars[-1]["usage"] != transform_content_to_ruby_rich_text(formatted_grammars[-1]["usage"]):
            empty_usage.append(formatted_grammars[-1])
    except Exception:
        print(grammar)
        print(content)
        print(detail)
        raise

print(formatted_grammars[0])
with open("grammars_N1-N5.json", "w", encoding="utf-8") as fp:
    fp.write(json.dumps(formatted_grammars, ensure_ascii=False, indent=4))

In [ ]:
for detail in empty_usage:
    print(detail["usage"])
    print(transform_content_to_ruby_rich_text(detail["usage"]))
    print("=" * 32)

In [ ]:
# 日语平假名unicode编码范围：\u3040-\u309F
# 日语片假名unicode编码范围：\u30A0-\u30FF

In [ ]:
content = "私の料理を一口食べるなり 、父は変な顔をして席を立ってしまった。"
get_md5(content)

In [ ]:
import os
import shutil
from zipfile import ZipFile


def extract_epub_images(epub_path, output_folder):
    with ZipFile(epub_path, 'r') as epub_file:
        for file_name in epub_file.namelist():
            if file_name.endswith(('.jpg', '.jpeg', '.png', '.gif')):
                output_file_path = os.path.join(output_folder, os.path.basename(file_name))
                with open(output_file_path, 'wb') as output_file:
                    output_file.write(epub_file.read(file_name))

    print("Image extraction completed.")


# 示例用法
epub_path = japanese_grammar.book_folder  / "超值白金版.蓝宝书大全集：新日本语能力考试N1-N5文法详解（最新修订版） (许小明) (Z-Library).epub" # 替换为实际的EPUB文件路径
output_folder = japanese_grammar.book_folder / "temp"  # 替换为实际的输出文件夹路径

extract_epub_images(epub_path, output_folder)

In [ ]:
from fpdf import FPDF
import os


def merge_jpg_to_pdf(folder_path, output_path):
    pdf = FPDF()

    for file_name in os.listdir(folder_path):
        if file_name.endswith(".jpg"):
            file_path = os.path.join(folder_path, file_name)
            pdf.add_page()
            pdf.image(file_path, x=0, y=0, w=210)  # 设置图片大小为A4纸尺寸

    pdf.output(output_path, "F")
    print("PDF merging completed.")


# 示例用法
folder_path = japanese_grammar.book_folder / "temp"  # 替换为实际的输出文件夹路径
output_path = japanese_grammar.book_folder / "file.pdf"  # 替换为实际的输出文件夹路径

merge_jpg_to_pdf(folder_path, output_path)

In [ ]:
!python -m pip install fpdf

In [ ]:
import time
import os
from tqdm import tqdm
import re
import sys

kana_meaning_pattern = re.compile(r"^[△\u2460-\u2470\s]*")
rich_text_pattern = re.compile(r"<ruby>(.*?)<rt>.*?</rt></ruby>")
RE_WHITESPACES_PATTERN = re.compile(r"\s+")

with open("grammars_N1-N5.json", "r", encoding="utf-8") as fp:
    formatted_grammars = json.loads(fp.read())

total_sentence_amount = sum([len(grammar["example"]) for grammar in formatted_grammars])
print(f"Total sentence amount: {total_sentence_amount}")

for grammar in tqdm(formatted_grammars, file=sys.stdout):
    for sentence in grammar["example"]:
        content = sentence["content"]
        content = kana_meaning_pattern.sub("", content)
        content = rich_text_pattern.sub(r"\1", content)
        content = RE_WHITESPACES_PATTERN.sub("", content)

        content_md5 = get_md5(content)
        audio_path = f"japanese_grammar/audio/{content_md5}.mp3"

        text_to_audio.generate_audio([
            (None, content),
        ], audio_path, log=tqdm.write)


In [ ]:
grammar_content_pattern = re.compile(r"^\d? ?[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF  ()]+")

grammar_content_pattern.findall("~ 際 (に)")

In [ ]:
content = "ここに車を止められるのは、許可をもらっている人（a だけ　b に限り）です。"
content = re.sub("\s+", " ", content)
print(content)

question_pattern = re.compile(r"[（(](?:[\da-n]\s*[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]+\s*)+[)）]")
print(question_pattern.findall(content))

In [ ]:
re.findall(r"(?:[\da-n]\s*[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]+\s*)+", "a だけ b に限り")

In [ ]:
question_pattern = re.compile(r"[(（](?:[\da-n]\s*[~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF]+\s*)+[）)]*\?")
print(question_pattern.findall("こちらの会議室をご利用になる際は、 受付で必要事項をご記入ください。"))
print(question_pattern.findall("ここに車を止められるのは、許可をもらっている人（a だけ　b に限り）です。"))

In [ ]:
import pandas as pd
import re

grammar_content_pattern = re.compile(r"^\d [~\u3040-\u309F\u30A0-\u30FF\u4E00-\u9FFF  ()]+")
sentence_content_pattern = re.compile(r'([\u2460-\u2470])\s*')
invalid_line_pattern = re.compile(r'^[\d\s]+$')

example_sentences = []
with open("CS_Word_20240403_23.41.17.txt", "r", encoding="utf-8") as fp:
    matched = False
    sentence = None
    last_line = ""
    for line in fp.readlines():
        line = line.strip()
        if len(line) < 3 or invalid_line_pattern.match(line.strip()):
            continue

        if grammar_content_pattern.match(line):
            print(line)

In [ ]:
import re
from collections import defaultdict

law_name_pattern = re.compile(r"^[一二三四五六七八九十]、.+$")
law_content_pattern = re.compile(r"^第([零一二三四五六七八九十百]+)条 .+$")

laws_to_answers = defaultdict(lambda : defaultdict(dict))

with open(
    "C:/Users/32740/Downloads/福建省幼师考试法律法规填空题答案.txt", "r", encoding="utf-8"
) as fp:
    lines = fp.readlines()

law_name = ""
law_to_amount = {}
count = 0
for line in lines:
    line = line.strip().strip("\ufeff")
    if not line:
        continue

    if law_name_pattern.match(line):
        if law_name:
            law_to_amount[law_name] = count

        law_name = line[2:]
        count = 0
        continue

    matched = law_content_pattern.findall(line)
    content_idx = chinese_to_number(matched[0])
    items = line.split()
    answers = items[1].split("；")
    laws_to_answers[law_name][content_idx] = answers
    count += 1

print(law_to_amount)
len(laws_to_answers["《幼儿园管理条例》"])

In [ ]:
import re
from collections import defaultdict

invalid_pattern = re.compile(r"^[\d]+$")
law_name_pattern = re.compile(r"^[一二三四五六七八九十]、.+$")
chapter_name_pattern = re.compile(r"^第[一二三四五六七八九十]+章 .+$")
law_content_pattern = re.compile(r"^第([零一二三四五六七八九十百]+)条 .+$")
trim_pattern = re.compile(
    r"(?:^[\s\ufeff]*(?:2004)?[\s\ufeff]*|[\s\ufeff]*(?:2004)?[\s\ufeff]*$)"
)
invalid_keywords = {
    "闽试教师幼儿园教育法律法规重点挖空练习",
}

with open(
    "C:/Users/32740/Downloads/福建24届教育法律法规精选重点挖空练习+幼儿园题目.txt",
    "r",
    encoding="utf-8",
) as fp:
    lines = fp.readlines()

law_name = ""
chatper_name = ""
content = []
laws = defaultdict(lambda: defaultdict(list))
law_to_numbers = defaultdict(set)

for idx, line in enumerate(lines):
    line = line.strip()
    line = trim_pattern.sub("", line)
    line = line.strip()
    if not line or line in invalid_keywords:
        continue

    if invalid_pattern.match(line):
        print(line)
        continue

    if law_name_pattern.match(line):
        if content:
            laws[law_name][chatper_name].append((content_idx, "".join(content)))
            content = []

        law_name = line[2:]
        chatper_name = ""
        # print(line)
        continue

    if chapter_name_pattern.match(line):
        if content:
            laws[law_name][chatper_name].append((content_idx, "".join(content)))
            content = []

        chatper_name = line
        continue

    if matched := law_content_pattern.findall(line):
        if content:
            laws[law_name][chatper_name].append((content_idx, "".join(content)))
            
        content_idx = chinese_to_number(matched[0])
        content = [line]
        continue

    # print(line)

    if not chatper_name:
        # print(line)
        continue

    content.append(line)


if content:
    laws[law_name][chatper_name].append((content_idx, "".join(content)))
    content = []

law_to_content_number = {
    "《中华人民共和国教育法》（2021年修正）": 86,
    "《中华人民共和国教师法》（2009年修正）": 39,
    "《中华人民共和国未成年人保护法》（2020年修订）": 129,
    "《幼儿园管理条例》": 29,
    "《教师资格条例》": 23,
    "《幼儿园工作规程》": 66,
    "《学生伤害事故处理办法》（2010年修正）": 40,
    "《中小学幼儿园安全管理办法》": 64,
    "《中华人民共和国家庭教育促进法》": 55,
}

print(len(laws.keys()))

# for law_name, law_content in laws.items():
#     content_amount = sum(map(len, law_content.values()))
#     if content_amount != law_to_content_number[law_name]:
#         print(
#             f"{law_name} expected {law_to_content_number[law_name]} but got {content_amount}"
#         )
#         # print(
#         #     set(range(1, law_to_content_number[law_name] + 1))
#         #     - law_to_numbers[law_name]
#         # )
#     for chatper_name in law_content.keys():
#         print(f"{law_name} - {chatper_name}")
#     print("=" * 32)

print(content)
print(laws["《中华人民共和国教育法》（2021年修正）"]["第一章 总则"])
print(laws["《中华人民共和国教育法》（2021年修正）"]["第二章 教育基本制度"])
print(laws["《中华人民共和国教师法》（2009年修正）"])

data = [
    {
        "law_name": law_name,
        "chatper_name": chatper_name,
        "idx": item[0],
        "content": item[1],
        "answer": json.dumps(laws_to_answers[law_name][item[0]] or [], ensure_ascii=False),
    }
    for law_name, law_content in laws.items()
    for chatper_name, lines in law_content.items()
    for item in lines
]

df = pd.DataFrame(data, dtype=str).fillna("")
print(df[df.law_name == "《中华人民共和国教育法》（2021年修正）"].chatper_name.unique())
df.to_csv("C:/Users/32740/Downloads/福建省幼师考试法律法规填空题.csv", index=False)

In [ ]:
import pandas as pd
import json

blank_pattern = re.compile(r"\s{2,}")
single_blank_pattern = re.compile(r"(?:\s+|_+)")

missing_laws = defaultdict(int)
df = pd.read_csv(
    "C:/Users/32740/Downloads/福建省幼师考试法律法规填空题.csv", dtype=str
).fillna("")

to_skip = {
    "《中华人民共和国教育法》（2021年修正）": 13,
    "《中华人民共和国未成年人保护法》（2020年修订）": 28,
    "《幼儿园工作规程》": 17,
}

for index in df.index:
    detail = df.loc[index].to_dict()
    answer = json.loads(detail["answer"])
    if not answer:
        continue
    content = detail["content"]

    first_blank_idx = content.find(" ")
    prefix = content[:first_blank_idx+1]
    postfix = content[first_blank_idx+1:]
    # matched = blank_pattern.findall(detail["content"])
    matched = single_blank_pattern.findall(postfix) 
    for idx, keyword in enumerate(matched):
        postfix = postfix.replace(keyword, "{{" + f"c{idx+1}::{answer[idx]}" + "}}", 1)

    content = prefix + postfix
    df.loc[index, "anki_question"] = content

df[df.answer != "[]"].to_csv("C:/Users/32740/Downloads/anki_福建省幼师考试法律法规填空题.csv", index=False)
missing_laws

In [ ]:
laws = list(laws_to_answers)
print(laws)
print(list(map(lambda law: len(laws_to_answers[law]), laws_to_answers)))
print(sum(map(lambda law: len(laws_to_answers[law]), laws_to_answers)))
print(df[df.answer != "[]"].shape)

{
    f"{law}_{answer_idx}"
    for law in laws_to_answers
    for answer_idx in laws_to_answers[law]
} - {
    f"{df.loc[index, 'law_name']}_{df.loc[index, 'idx']}"
    for index in df.index
    if df.loc[index, "answer"] != "[]"
}

In [ ]:
import requests

cookies = {
    'BAIDU_SSP_lcr': 'https://www.google.com/',
    'ASPSESSIONIDQUBCBADR': 'FIDPCAEAJJDMLFBCJLBCLOCK',
    'Hm_lvt_5a5e1257cebb73bd6a288064324f215b': '1718214462',
    'Hm_lpvt_5a5e1257cebb73bd6a288064324f215b': '1718214509',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en,zh-CN;q=0.9,zh;q=0.8',
    'Cache-Control': 'no-cache',
    'Connection': 'keep-alive',
    # 'Cookie': 'BAIDU_SSP_lcr=https://www.google.com/; ASPSESSIONIDQUBCBADR=FIDPCAEAJJDMLFBCJLBCLOCK; Hm_lvt_5a5e1257cebb73bd6a288064324f215b=1718214462; Hm_lpvt_5a5e1257cebb73bd6a288064324f215b=1718214509',
    'Pragma': 'no-cache',
    'Referer': 'https://www.google.com/',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'cross-site',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Google Chrome";v="125", "Chromium";v="125", "Not.A/Brand";v="24"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
}

# response = requests.get('https://www.waizi.org.cn/doc/111308.html', 
response = requests.get('https://www.waizi.org.cn/doc/251916.html', 
# cookies=cookies, 
headers=headers,)
html_doc = response.content

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_doc, 'html.parser')
text_container = soup.find(id='text')


In [ ]:
start = False

for child in text_container.find_all("p"):
    content = child.text.strip()
    if content == "第一章　总　　则":
        start = True

    if start:
        print(child.text)

    if content.endswith("第一百三十二条　本法自2021年6月1日起施行。"):
        start = False

In [ ]:
import requests

cookies = {
    'wdcid': '7caa5ff5ee26505f',
    'wdlast': '1718218088',
    'wdses': '51460f53be054a9c',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'Accept-Language': 'en,zh-CN;q=0.9,zh;q=0.8',
    'Cache-Control': 'no-cache',
    # 'Cookie': 'wdcid=7caa5ff5ee26505f; wdlast=1718218088; wdses=51460f53be054a9c',
    'Pragma': 'no-cache',
    'Proxy-Connection': 'keep-alive',
    'Referer': 'https://www.google.com/',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36',
}

response = requests.get(
    'http://www.moe.gov.cn/srcsite/A02/s5911/moe_621/201602/t20160229_231184.html',
    # cookies=cookies,
    headers=headers,
    verify=False,
)
html_doc = response.content

In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup(html_doc, 'html.parser')
text_container = soup.find(id='downloadContent')


start = False

for child in text_container.find_all("p"):
    content = child.text.strip()
    if content.startswith("第一章"):
        start = True

    if start:
        print(child.text)

In [ ]:
import os
import requests
import base64

# Configuration https://shibainu.openai.azure.com/
GPT4V_KEY = "f7d7d95881d440dda5f275c5574f04d0"
# IMAGE_PATH = "YOUR_IMAGE_PATH"
# encoded_image = base64.b64encode(open(IMAGE_PATH, "rb").read()).decode("ascii")
headers = {
    "Content-Type": "application/json",
    "api-key": GPT4V_KEY,
}

# Payload for the request
payload = {
    "messages": [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": "You are an AI assistant that helps people find information.",
                }
            ],
        }
    ],
    "temperature": 0.7,
    "top_p": 0.95,
    "max_tokens": 800,
}

GPT4V_ENDPOINT = "https://shibainu.openai.azure.com/openai/deployments/gpt-4o/chat/completions?api-version=2024-02-15-preview"

# Send request
try:
    response = requests.post(GPT4V_ENDPOINT, headers=headers, json=payload)
    response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
except requests.RequestException as e:
    raise SystemExit(f"Failed to make the request. Error: {e}")

# Handle the response as needed (e.g., print or process)
print(response.json())

In [ ]:
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    openai_api_version="f7d7d95881d440dda5f275c5574f04d0",
    azure_deployment="gpt-4o",
    temperature=0,
)

In [ ]:
from IPython.display import Image, display, Audio, Markdown
import base64

IMAGE_PATH = "C:/Users/32740/Downloads/bk13.png"

# Preview image for context
display(Image(IMAGE_PATH))


# Open the image file and encode it as a base64 string
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode("utf-8")


base64_image = encode_image(IMAGE_PATH)

messages = [
    {
        "role": "system",
        "content": "You are a helpful assistant that responds in Markdown.",
    },
    {
        "role": "user",
        "content": [
            {"type": "text", "text": "Describe the images as an alternative text"},
            {
                "type": "image_url",
                "image_url": {"url": f"data:image/png;base64,{base64_image}"},
            },
        ],
    },
]
ai_message = llm.invoke(messages)
print(ai_message.content)

In [ ]:
from langchain_openai import AzureChatOpenAI

llm = AzureChatOpenAI(
    azure_deployment="gpt-4o",
    api_version="2024-02-15-preview",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
from langchain import LLMChain, PromptTemplate

sentence_analyzing_prompt_template = """
    analyze the following Japanese sentence and using Chinese to explain the used grammar and words:
    requirement 1: you should use Chinese to reply.
    ---
    {content}
    """
sentence_analyzing_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(sentence_analyzing_prompt_template),
    output_key="analysis",
)
content = "お客様あっての仕事ですから、いつもご来店いただくお客様には感謝しております。"
result: str = sentence_analyzing_chain.run(
    {
        "content": content,
    }
)
print(result)

In [ ]:
import markdown

# 示例 Markdown 文本
markdown_text = """
# 这是一个标题

这是一个段落，其中包含 **粗体** 和 *斜体* 文本。

- 列表项 1
- 列表项 2
- 列表项 3

[这是一个链接](https://www.example.com)

```python
print("这是一个代码块")
```
"""


html = markdown.markdown(markdown_text)

In [ ]:
print(html)

In [ ]:
from PyPDF2 import PdfReader, PdfWriter

def crop_pdf(input_pdf_path, crop_ratios, output_pdf_path):
    """
    Crop each page of a PDF file based on the given ratios and save the output to a new file.

    :param input_pdf_path: Path to the input PDF file.
    :param crop_ratios: A tuple of four ratios (left_ratio, top_ratio, right_ratio, bottom_ratio).
                        Each ratio represents the fraction to crop from the corresponding side.
    :param output_pdf_path: Path to save the cropped PDF file.
    """
    left_ratio, top_ratio, right_ratio, bottom_ratio = crop_ratios

    # Read the input PDF
    reader = PdfReader(input_pdf_path)
    writer = PdfWriter()

    # Iterate through all pages and crop each one
    for page in reader.pages:
        media_box = page.mediabox

        # Get original dimensions
        orig_left = float(media_box.left)
        orig_bottom = float(media_box.bottom)
        orig_right = float(media_box.right)
        orig_top = float(media_box.top)

        # Calculate new boundaries
        new_left = orig_left + (orig_right - orig_left) * left_ratio
        new_right = orig_right - (orig_right - orig_left) * right_ratio
        new_bottom = orig_bottom + (orig_top - orig_bottom) * bottom_ratio
        new_top = orig_top - (orig_top - orig_bottom) * top_ratio

        # Set the new dimensions
        page.mediabox.lower_left = (new_left, new_bottom)
        page.mediabox.upper_right = (new_right, new_top)

        # Add the cropped page to the writer
        writer.add_page(page)

    # Write the cropped pages to a new file
    with open(output_pdf_path, 'wb') as out_file:
        writer.write(out_file)

    print(f"Cropped PDF saved to {output_pdf_path}")

# Example usage:
# crop_pdf("input.pdf", (0.1, 0.1, 0.1, 0.1), "output.pdf")



# 示例用法
input_path = 'G:/BaiduNetdiskDownload/日语真题（1992-2023年12月）/日语N1真题/日语N1真题/【03】N1 2020年-2023年12月/2023.12/2023年12月N1真题.pdf'
ratios = (0.05, 0.05, 0.05, 0.075)  # 裁剪比例示例  bottom, left, top, right 
output_path = 'C:/Users/32740/Downloads/output.pdf'

crop_pdf(input_path, ratios, output_path)


In [ ]:
import re

def parse_questions(content):
    questions = []
    current_question = {}
    options = []

    for line in content.strip().split('\n'):
        line = line.strip()

        if line:
            if line.isdigit():
                if current_question:
                    current_question['options'] = options
                    questions.append(current_question)
                    current_question = {}
                    options = []
                current_question['number'] = int(line)
            elif re.match(r'\d+\s+', line):
                option_num, option_text = line.split(maxsplit=1)
                options.append(option_text)
            else:
                if 'content' in current_question:
                    current_question['content'] += ' ' + line
                else:
                    current_question['content'] = line

    if current_question:
        current_question['options'] = options
        questions.append(current_question)

    return questions

In [ ]:
from docx import Document

def read_docx(file_path):
    doc = Document(file_path)
    full_text = []
    for paragraph in doc.paragraphs:
        full_text.append(paragraph.text)
    return '\n'.join(full_text)



# 示例用法
file_path = 'C:/Users/32740/Downloads/CS_Word_20240815_02.15.13.docx'
text = read_docx(file_path)
# print(text)
parse_questions(text)

In [ ]:
parse_questions(text)

In [ ]:
pattern = re.compile(r"問題\s*\d")
for line in text.split("\n"):
    if pattern.search(line):
        print(line)

In [ ]:
print(text)

In [ ]:
pattern = re.compile(r"\d+\s\D*")

file_path = "C:/Users/32740/Downloads/CS_Word_20240815_02.15.13.docx"
content = read_docx(file_path)

last_line = ""
all_result = []
reading_area = False
lines = []
for line in content.split("\n"):
    line = line.strip()
    if not line:
        continue

    if line.split()[0] == "問題1":
        if not reading_area:
            reading_area = True

    if line.split()[0] == "問題8":
        break

    if not reading_area:
        continue

    lines.append(line)

    # if line[:2] == "問題":
    #     continue

    # if line.isdigit():
    #     last_line = line
    #     continue

    # if last_line.isdigit():
    #     line = last_line + " " + line

    # matched = pattern.findall(line)
    # last_line = line

    # if not matched:
    #     print(line)
    # print(matched)

    # all_result.extend(matched)

In [ ]:
print("\n".join(lines))

In [ ]:
digit_pattern = re.compile(r"^\d")

for index, line in enumerate(lines):
    if line[:2] == "問題":
        lines[index] = ""
        continue

    if not digit_pattern.match(line):
        last_line = lines[index - 1]
        if not last_line:
            lines[index] = ""
            continue

        print(f"last_line: {last_line}")
        print(f"line: {line}")

In [60]:
import os
import PyPDF2


def get_pdf_page_count(file_path):
    with open(file_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        return len(reader.pages)


folder_path = "G:/Code/aki_api/data"
for file in os.listdir(folder_path):
    file_path = os.path.join(folder_path, file)
    if os.path.isfile(file_path) and file_path.endswith(".pdf"):
        page_count = get_pdf_page_count(file_path)
        print(f"{file}: {page_count}")

2022年12月N1真题.pdf: 22


incorrect startxref pointer(1)


2022年7月N1真题完整版.pdf: 34
2023年12月N1真题.pdf: 52
2023年7月N1真题 （13页）.pdf: 13
N1 2020年12月真题+答案+详解+听力原文.pdf: 31
N1 2021年12月真题+答案+详解+听力原文.pdf: 33
N1 2021年7月真题+答案+详解+听力原文.pdf: 31


In [ ]:
content = """
36 この花火大会は、日本の夏を語る    有名だ。
1. うえで
2. として
3. 欠かせない
4.イベント

37. 子供にいろいろなことを習わせたいという親の気持ちはよくわかるが、 子供の遊ぶ時間を奪って    ないと思う。
1.必要は
2. やらせる
3.まで
4. 無理に

38 A「おとといは大雨、 昨日は車のパンク。 旅行に来てからトラブル続きで嫌になるよね。 」 B 「本当だよ。最終日の     。」 
1.何も
2.今日こそ
3.ように
4. 起きません

39 2025年には介護を必要とする高齢者が著しく増加することから、多くの専門家が 「介護施設職員の給与を引き上げる    と指摘する。
1.などして
2. 緊急の課題
3.人材を確保することが
4. 介護に携わる

40川北市出身の画家平(たいら) (かず) 明 (あき)の    先週20日に完成した。
1.建設を進めて
2. 川北市が
3. 記念美術館が
4. 業績を後世に伝えようと


36 そんな簡単なこと、 わざわざあなたに
1.もらう
2. ない
3. 説明して
4. までも

37 ちょっと考えれば、さっきの話が冗談    単純な彼は簡単に信じてしまった。
1. わかるだろう
2. だって
3.に
4. ことくらい

38 彼はとても優秀で成績が学年の上位に入っている    真面目で好感が持てる。
1.授業に取り組む
2. のみならず
3. 姿勢そのものも
4.ことが多い

39. 全国高校バスケットボール大会で、 惜しくも    しばらくの間、ぼう然としていた。
1. 優勝を逃した
2. 控え室に戻っても
3.選手たちは
4. あと一歩というところで

40. 私が接客するにあたって    自分は何をすべきかということだ。
1. 常に考えているのは
2. 何であって
3. それに応えるために
4.お客様が求めていることは

36.ステージに     上がった。
1. なり
2. 登場する
3.歓声が
4.歌手が

37. 彼はこの映画で、 純料で不器用な    までに演じきった。
1. その確かな
2. 主人公を
3. 見事な
4. 表現力で


38. あの人のことが好きて    いいと思う。
1. 忘れられないんだったら
2. 何度でも気持ちを伝えたら
3. 一度断られたぐらいで
4. あきらめたりしないで


39.他人のことは     人間というものだ。
1. 自分のこととなると
2. そう簡単にはいかず
3. 感情的になってしまうのが
4. 冷静に見られるのに


40.介護の現場における    そのうち必ず限界が来る。 介護ロボットの普及が期待される。
1. このまま
2. 人の手のみに頼っていては
3. 人手不足が深刻している
4.中で


36、(インクビューで) 「私が30年間歌手を続けてこられたのは、ファンの方の支えがあったからです。    歌い続けたいと思っています。」
1 限り
2 ファンの皆さんが
3 応援してくれる
4 いる

37 S 市が 18歳以上の    運動不足を感じていることがわかった。
1 意識調査を行ったところ
3 多くの人が
2 市民を対象に
4 運動に関する


38、 子供のころ、 母はしつけに厳しくて、 私はそれが嫌だった。 しかし、 母が    今ならわかる。
1 親になった
2 私のことを
3 思えばこそだったのだと
4 厳しかったのは


39、 去年    若者の間で流行している。
1 レインコートは
2 機能性もさることながら
3 M社から発売された
4 そのかわいらしいデザインが話題となり


40、 「未来のものづくりコンテスト」 は、     今年で20回目を迎える。
1 子供たちに
2 コンテストで
3 ものづくりの面白さを感じてもらおうと
4 ABC 社が創立50周年を機に始めた

36) 秋の初めてのこの時期は、真夏の戻った    気温の差が大きくて体調を崩しやすいので、注意が必要だ。
1 日もあり
2 日もあれば
3 ひんやりとした
4 かのような

37) 歴史的価値が高いとされる旧白本小学校の校舎を始めて見たが、 とても    驚いた。
1 100年前に
2 現代的なデザインに
3 建てられたものとは
4 思えないほどの

38)くしゃみや鼻水といった症状が現れる花粉症は、風邪と    ケースもあるそうだ。
1  自覚がない
2 自分は花粉症だという
3 症状が似ている
4 こともあって

39) 高校卒業後の進路について両親に反対されたが、誰に何と    思う。
1 思える道を
2 信じて進みたいと
3 言われようとも
4 自分がこれだと

40) 企業の海外進出が成功するかどうかは、    といってもいいだろう。
1 優秀な人材を確保できる
2 にかかっている
3 か否か
4 その国の事情をよく知る

36 スピーチやプレゼンテーションにおいて、ジェスチャーを使いながら話すのは    印象を悪くすることもある。
1 あまりに
2 効果的である反面
3 かえって
4 大きすぎるジェスチャーは

37 今や、書類や衣料品だけでなく生鮮食品も、インターネットへの    時代である。
1 自宅にいながらにして
2 手軽に購入できる
3 環境があれば
4 アクセスが可能な

38 よく似た昆虫の判別は大変難しいという。 中には、 かなり    場合もあるそうだ。
1 判別が難しい
2 昆虫学者でさえも
3 経験を積んだ
4 違うほど

39 今回の    気持ちの方が大きかった。
1 転職にあたり
2 うそになるが
3 少しも不安がなかったといえば
4 新たなことに挑戦できてうれしいという

40 北山市は    のどかなところだった。
1 人口10万人を超える都市となったが
2 北山駅周辺以外にはほとんど何もない
3 今でこそ
4 30年前までは

36 当社が先月発売したパソコンについて、 印刷の    ことが判明しました。
1 マニュアルに誤りがある
2 設定ができないとの
3 問い合わせが複数寄せられ
4 調査したところ

37 このフライパンは、さすが    とても使いやすい。
1 森さんが
2 だけあって
3 勧める
4 料理が上手な

38 最後に見た映画が    映画を見ていない。
1 ほど
2 何だったのかも
3 思い出せない
4 久しく

39 学生から提出された論文の中に面白いものがあった。 私が    これまでにない。
1 知る限りでは
2 論文は
3 分析している
4 このアプローチで

40 このレシピ本は、ふだん料理をしない人でも    評判になっているそうだ。
1 若い人の間で
2 調理するだけで
3 レシピに沿って
4 簡単に本格的な料理が作れると
"""